# Setup/Test Environment:

In [1]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [2]:
import sys
sys.executable

'/miniconda/bin/python'

In [3]:
%env TARGET=10.10.133.181

env: TARGET=10.10.133.181


In [4]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
    input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

/bin/sh: 1: docker: not found


# Initial Nmap:

In [7]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.10.133.181


In [8]:
!ping -c 5 $TARGET

/bin/sh: 1: ping: not found


# Quick Scan:

In [9]:
!nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:35 UTC
Initiating ARP Ping Scan at 02:35
Scanning 10.10.133.181 [1 port]
Completed ARP Ping Scan at 02:35, 0.06s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:35
Completed Parallel DNS resolution of 1 host. at 02:35, 0.00s elapsed
Initiating SYN Stealth Scan at 02:35
Scanning ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181) [1000 ports]
Discovered open port 135/tcp on 10.10.133.181
Discovered open port 139/tcp on 10.10.133.181
Discovered open port 80/tcp on 10.10.133.181
Discovered open port 389/tcp on 10.10.133.181
Discovered open port 88/tcp on 10.10.133.181
Discovered open port 3268/tcp on 10.10.133.181
Discovered open port 3389/tcp on 10.10.133.181
Discovered open port 53/tcp on 10.10.133.181
Discovered open port 445/tcp on 10.10.133.181
Discovered open port 593/tcp on 10.10.133.181
Discovered open port 464/tcp on 10.10.133.181
Discovered open port 3269/tcp on 10.10.133.181
Discovered 

# All Ports:

In [10]:
!nmap -p- -sV -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:36 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 02:36
Scanning 10.10.133.181 [1 port]
Completed ARP Ping Scan at 02:36, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:36
Completed Parallel DNS resolution of 1 host. at 02:36, 0.00s elapsed
Initiating SYN Stealth Scan at 02:36
Scanning ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181) [65535 ports]
Discovered open port 445/tcp on 10.10.133.181
Discovered open port 139/tcp on 10.10.133.181
Discovered open port 53/tcp on 10.10.133.181
Discovered open port 135/tcp on 10.10.133.181
Discovered open port 80/tcp on 10.10.133.181
Discovered open port 3389/tcp on 10.10.133.181
Increasing send delay for 10.10.133.181 from 0 to 5 due to 25 out of 83 dropped probes since last increase.
Discovered open port 49664/tcp on 10.10.133.181
SYN Stealth Scan Timing: About 9.06% done; ETC: 02:41 (0:05:11 remaining)
Discovered open port

In [11]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [12]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=53,80,88,135,139,389,445,464,593,636,3268,3269,3389,5985,9389,47001,49664,49665,49666,49669,49670,49675,49676,49677,49680,49685,49697,49818


# No Ping:

In [13]:
!nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:42 UTC


Host discovery disabled (-Pn). All addresses will be marked 'up' and scan times will be slower.


NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 02:42
Scanning 10.10.133.181 [1 port]
Completed ARP Ping Scan at 02:42, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:42
Completed Parallel DNS resolution of 1 host. at 02:42, 0.00s elapsed
Initiating SYN Stealth Scan at 02:42
Scanning ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181) [1000 ports]
Discovered open port 135/tcp on 10.10.133.181
Discovered open port 80/tcp on 10.10.133.181
Discovered open port 445/tcp on 10.10.133.181
Discovered open port 53/tcp on 10.10.133.181
Discovered open port 3389/tcp on 10.10.133.181
Discovered open port 139/tcp on 10.10.133.181
Increasing send delay for 10.10.133.181 from 0 to 5 due to 26 out of 86 dropped probes since last increase.
Discovered open port 3268/tcp on 10.10.133.181
Discovered open port 3269/tcp on 10.10.133.181
Discovered open port 88/tcp on 10.10.133.181
Discovered open port 636/tcp on 10.10.133.181
Discovered open port 5

In [14]:
if not PING:
    !nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [15]:
if PING:
    !nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:42 UTC
NSE: Loaded 123 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 02:42
Completed NSE at 02:42, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 02:42
Completed NSE at 02:42, 0.00s elapsed
Initiating ARP Ping Scan at 02:42
Scanning 10.10.133.181 [1 port]
Completed ARP Ping Scan at 02:42, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 02:42
Completed Parallel DNS resolution of 1 host. at 02:42, 0.00s elapsed
Initiating SYN Stealth Scan at 02:42
Scanning ip-10-10-133-181.eu-west-1.compute.internal (10.10.133.181) [28 ports]
Discovered open port 135/tcp on 10.10.133.181
Discovered open port 3389/tcp on 10.10.133.181
Discovered open port 445/tcp on 10.10.133.181
Discovered open port 80/tcp on 10.10.133.181
Discovered open port 139/tcp on 10.10.133.181
Discovered open port 53/tcp on 10.10.133.181
Discovered open port 49669

In [16]:
if PING:
    !nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:45 UTC
NSE: Loaded 419 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 02:45
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE Timing: About 95.88% done; ETC: 02:46 (0:00:01 remaining)
Completed NSE at 02:46, 40.00s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 02:46
Completed NSE at 02:46, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan.
Initiati

In [17]:
if PING:
    !nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-05 02:49 UTC
NSE: Loaded 419 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 02:49
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE Timing: About 98.98% done; ETC: 02:50 (0:00:00 remaining)
Completed NSE at 02:50, 40.08s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 02:50
Completed NSE at 02:50, 0.00s elapsed
NSE: Starting runlevel 3 (of 3) scan.
Initiati

In [ ]:
from getpass import getpass
import os

if PING:
    !echo { getpass() } | sudo -S nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !echo { getpass() } | sudo -S nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
# UDP Scan, this takes forever...

# from getpass import getpass

# !echo { getpass() } > tmp.txt
# !cat tmp.txt | sudo -S nmap -sU --open -oA scans/udp/udp_scan $TARGET
# !cat tmp.txt | sudo -S nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET
# !rm -rf tmp.txt